In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 22.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [20, 6]

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('data/부동산데이터분석_아파트단지별정보_서울_변수TF_13_변수작업완료.xlsx')
df

,법정동주소,단지명,단지코드,도로명주소,시군구,동리,거래허가제TF,아파트TF,주상복합TF,도시형주택TF,...,지역난방TF,복도식TF,혼합형TF,계단식TF,메이저시공사TF,조합시행TF,철근철골구조TF,홈네트워크TF,관리시설수,부대복리시설합
0,서울특별시 마포구 아현동 799 공덕자이(임대),공덕자이(임대),A10027951,서울특별시 마포구 마포대로26길 22,마포구,아현동,0,1,0,0,...,0,1,0,0,1,1,0,1,5,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소"
1,서울특별시 성북구 길음동 1285-5 길음뉴타운7단지제2,길음뉴타운7단지제2,A13611012,서울특별시 성북구 길음로13길 22,성북구,길음동,0,1,0,0,...,0,1,0,0,1,0,0,0,4,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
2,서울특별시 성북구 길음동 1284-9 길음뉴타운8단지제2,길음뉴타운8단지제2,A13611010,서울특별시 성북구 길음로 33,성북구,길음동,0,1,0,0,...,0,1,0,0,1,0,0,0,4,"관리사무소, 노인정, 주민공동시설, 어린이놀이터"
3,서울특별시 마포구 도화동 564 마포동원베네스트,마포동원베네스트,A12170401,서울특별시 마포구 도화4길 41,마포구,도화동,0,1,0,0,...,0,0,0,1,0,0,0,0,4,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
4,서울특별시 강남구 청담동 23 청담2차현대아파트,청담2차현대아파트,A13510202,서울특별시 강남구 선릉로 748,강남구,청담동,1,1,0,0,...,1,0,0,1,1,0,0,0,2,"관리사무소, 어린이놀이터"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794,서울특별시 강동구 천호동 46 강동이루미,강동이루미,A13402004,서울특별시 강동구 양재대로123길 30,강동구,천호동,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN
2795,서울특별시 용산구 한남동 726-74 남산맨션,남산맨션,A54016001,NaN,용산구,한남동,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN
2796,서울특별시 구로구 항동 1-5 그린빌라,그린빌라,A15271602,서울특별시 구로구 연동로 287,구로구,항동,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN
2797,서울특별시 강서구 화곡동 1156 우장산휴먼빌,우장산휴먼빌,A15701005,NaN,강서구,화곡동,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN


In [3]:
df['단지명'] = df['단지명'].str.replace(r'\([^)]*\)', '', regex=True)
df['ID'] = '서울특별시' + ' ' + df['시군구'] + ' ' + df['동리'] + ' ' + df['단지명'].astype('str')
df.head()

,법정동주소,단지명,단지코드,도로명주소,시군구,동리,거래허가제TF,아파트TF,주상복합TF,도시형주택TF,...,복도식TF,혼합형TF,계단식TF,메이저시공사TF,조합시행TF,철근철골구조TF,홈네트워크TF,관리시설수,부대복리시설합,ID
0,서울특별시 마포구 아현동 799 공덕자이(임대),공덕자이,A10027951,서울특별시 마포구 마포대로26길 22,마포구,아현동,0,1,0,0,...,1,0,0,1,1,0,1,5,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소",서울특별시 마포구 아현동 공덕자이
1,서울특별시 성북구 길음동 1285-5 길음뉴타운7단지제2,길음뉴타운7단지제2,A13611012,서울특별시 성북구 길음로13길 22,성북구,길음동,0,1,0,0,...,1,0,0,1,0,0,0,4,"관리사무소, 노인정, 어린이놀이터, 자전거보관소",서울특별시 성북구 길음동 길음뉴타운7단지제2
2,서울특별시 성북구 길음동 1284-9 길음뉴타운8단지제2,길음뉴타운8단지제2,A13611010,서울특별시 성북구 길음로 33,성북구,길음동,0,1,0,0,...,1,0,0,1,0,0,0,4,"관리사무소, 노인정, 주민공동시설, 어린이놀이터",서울특별시 성북구 길음동 길음뉴타운8단지제2
3,서울특별시 마포구 도화동 564 마포동원베네스트,마포동원베네스트,A12170401,서울특별시 마포구 도화4길 41,마포구,도화동,0,1,0,0,...,0,0,1,0,0,0,0,4,"관리사무소, 노인정, 어린이놀이터, 자전거보관소",서울특별시 마포구 도화동 마포동원베네스트
4,서울특별시 강남구 청담동 23 청담2차현대아파트,청담2차현대아파트,A13510202,서울특별시 강남구 선릉로 748,강남구,청담동,1,1,0,0,...,0,0,1,1,0,0,0,2,"관리사무소, 어린이놀이터",서울특별시 강남구 청담동 청담2차현대아파트


In [8]:
dff = df[['ID','세대수','동당세대수','관리원당세대수','승강기당세대수','세대당주차수','지하주차비율','세대당cctv수','관리인원수','연간소독횟수','개별난방TF','중앙난방TF','지역난방TF','복도식TF','혼합형TF','계단식TF','홈네트워크TF','관리시설수','부대복리시설합']]
dff

,ID,세대수,동당세대수,관리원당세대수,승강기당세대수,세대당주차수,지하주차비율,세대당cctv수,관리인원수,연간소독횟수,개별난방TF,중앙난방TF,지역난방TF,복도식TF,혼합형TF,계단식TF,홈네트워크TF,관리시설수,부대복리시설합
0,서울특별시 마포구 아현동 공덕자이,201,100.500000,33.500000,9.571429,0.701493,1.000000,0.313433,6.00,0.0,1,0,0,1,0,0,1,5,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소"
1,서울특별시 성북구 길음동 길음뉴타운7단지제2,99,99.000000,34.615385,7.215743,0.767677,1.000000,0.181818,2.86,0.0,1,0,0,1,0,0,0,4,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
2,서울특별시 성북구 길음동 길음뉴타운8단지제2,120,120.000000,38.709677,9.836066,0.700000,1.000000,0.216667,3.10,0.0,1,0,0,1,0,0,0,4,"관리사무소, 노인정, 주민공동시설, 어린이놀이터"
3,서울특별시 마포구 도화동 마포동원베네스트,167,55.666667,23.857143,6.958333,1.000000,0.958084,0.365269,7.00,0.0,1,0,0,0,0,1,0,4,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
4,서울특별시 강남구 청담동 청담2차현대아파트,214,107.000000,19.454545,6.114286,0.878505,0.398936,0.116822,11.00,0.0,0,0,1,0,0,1,0,2,"관리사무소, 어린이놀이터"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794,서울특별시 강동구 천호동 강동이루미,74,0.000000,0.000000,74.000000,0.000000,0.000000,0.000000,0.00,NaN,0,0,0,0,0,1,0,0,NaN
2795,서울특별시 용산구 한남동 남산맨션,129,0.000000,0.000000,129.000000,0.000000,0.000000,0.000000,0.00,NaN,0,0,0,0,0,1,0,0,NaN
2796,서울특별시 구로구 항동 그린빌라,137,0.000000,0.000000,137.000000,0.000000,0.000000,0.000000,0.00,NaN,0,0,0,0,0,1,0,0,NaN
2797,서울특별시 강서구 화곡동 우장산휴먼빌,62,0.000000,0.000000,62.000000,0.000000,0.000000,0.000000,0.00,NaN,0,0,0,0,0,1,0,0,NaN


In [10]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2799 entries, 0 to 2798
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        2799 non-null   object 
 1   세대수       2799 non-null   int64  
 2   동당세대수     2799 non-null   float64
 3   관리원당세대수   2799 non-null   float64
 4   승강기당세대수   2799 non-null   float64
 5   세대당주차수    2799 non-null   float64
 6   지하주차비율    2799 non-null   float64
 7   세대당cctv수  2799 non-null   float64
 8   관리인원수     2799 non-null   float64
 9   연간소독횟수    2567 non-null   float64
 10  개별난방TF    2799 non-null   int64  
 11  중앙난방TF    2799 non-null   int64  
 12  지역난방TF    2799 non-null   int64  
 13  복도식TF     2799 non-null   int64  
 14  혼합형TF     2799 non-null   int64  
 15  계단식TF     2799 non-null   int64  
 16  홈네트워크TF   2799 non-null   int64  
 17  관리시설수     2799 non-null   int64  
 18  부대복리시설합   2603 non-null   object 
dtypes: float64(8), int64(9), object(2)
memory usage: 415.6+ KB


In [13]:
# count_com = []

for i in range(len(dff)):
    count_com = dff['부대복리시설합'].str.count(',') + 1

count_com

0       5.0
1       4.0
2       4.0
3       4.0
4       2.0
       ... 
2794    NaN
2795    NaN
2796    NaN
2797    NaN
2798    NaN
Name: 부대복리시설합, Length: 2799, dtype: float64

In [14]:
dff['부대복리시설수'] = count_com
dff.head()

,ID,세대수,동당세대수,관리원당세대수,승강기당세대수,세대당주차수,지하주차비율,세대당cctv수,관리인원수,연간소독횟수,개별난방TF,중앙난방TF,지역난방TF,복도식TF,혼합형TF,계단식TF,홈네트워크TF,관리시설수,부대복리시설합,부대복리시설수
0,서울특별시 마포구 아현동 공덕자이,201,100.500000,33.500000,9.571429,0.701493,1.000000,0.313433,6.00,0.0,1,0,0,1,0,0,1,5,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소",5.0
1,서울특별시 성북구 길음동 길음뉴타운7단지제2,99,99.000000,34.615385,7.215743,0.767677,1.000000,0.181818,2.86,0.0,1,0,0,1,0,0,0,4,"관리사무소, 노인정, 어린이놀이터, 자전거보관소",4.0
2,서울특별시 성북구 길음동 길음뉴타운8단지제2,120,120.000000,38.709677,9.836066,0.700000,1.000000,0.216667,3.10,0.0,1,0,0,1,0,0,0,4,"관리사무소, 노인정, 주민공동시설, 어린이놀이터",4.0
3,서울특별시 마포구 도화동 마포동원베네스트,167,55.666667,23.857143,6.958333,1.000000,0.958084,0.365269,7.00,0.0,1,0,0,0,0,1,0,4,"관리사무소, 노인정, 어린이놀이터, 자전거보관소",4.0
4,서울특별시 강남구 청담동 청담2차현대아파트,214,107.000000,19.454545,6.114286,0.878505,0.398936,0.116822,11.00,0.0,0,0,1,0,0,1,0,2,"관리사무소, 어린이놀이터",2.0


In [15]:
del dff['부대복리시설합']

In [17]:
dff.fillna(0, inplace=True)
dff

,ID,세대수,동당세대수,관리원당세대수,승강기당세대수,세대당주차수,지하주차비율,세대당cctv수,관리인원수,연간소독횟수,개별난방TF,중앙난방TF,지역난방TF,복도식TF,혼합형TF,계단식TF,홈네트워크TF,관리시설수,부대복리시설수
0,서울특별시 마포구 아현동 공덕자이,201,100.500000,33.500000,9.571429,0.701493,1.000000,0.313433,6.00,0.0,1,0,0,1,0,0,1,5,5.0
1,서울특별시 성북구 길음동 길음뉴타운7단지제2,99,99.000000,34.615385,7.215743,0.767677,1.000000,0.181818,2.86,0.0,1,0,0,1,0,0,0,4,4.0
2,서울특별시 성북구 길음동 길음뉴타운8단지제2,120,120.000000,38.709677,9.836066,0.700000,1.000000,0.216667,3.10,0.0,1,0,0,1,0,0,0,4,4.0
3,서울특별시 마포구 도화동 마포동원베네스트,167,55.666667,23.857143,6.958333,1.000000,0.958084,0.365269,7.00,0.0,1,0,0,0,0,1,0,4,4.0
4,서울특별시 강남구 청담동 청담2차현대아파트,214,107.000000,19.454545,6.114286,0.878505,0.398936,0.116822,11.00,0.0,0,0,1,0,0,1,0,2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794,서울특별시 강동구 천호동 강동이루미,74,0.000000,0.000000,74.000000,0.000000,0.000000,0.000000,0.00,0.0,0,0,0,0,0,1,0,0,0.0
2795,서울특별시 용산구 한남동 남산맨션,129,0.000000,0.000000,129.000000,0.000000,0.000000,0.000000,0.00,0.0,0,0,0,0,0,1,0,0,0.0
2796,서울특별시 구로구 항동 그린빌라,137,0.000000,0.000000,137.000000,0.000000,0.000000,0.000000,0.00,0.0,0,0,0,0,0,1,0,0,0.0
2797,서울특별시 강서구 화곡동 우장산휴먼빌,62,0.000000,0.000000,62.000000,0.000000,0.000000,0.000000,0.00,0.0,0,0,0,0,0,1,0,0,0.0


In [20]:
dff.to_excel('data/추가데이터.xlsx')

In [18]:
data = pd.read_excel('data/database.xlsx')
data.head()

,ID,거래금액(만원)_18/1,전세금(만원)_18/1,등락율18/1,전세가율18/1,인접역 수,전용면적,건축년도,Brand,BrandPower,...,인접역 수.17,전용면적.17,건축년도.17,Brand.17,BrandPower.17,구별인구.17,거래량18,cluster18,위도,경도
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,42166.7,4.162633,32.7,0,80,1987.0,0,0,...,0,80,1987.0,0,0,536710,3,4,37.483372,127.053437
1,서울특별시 강남구 개포동 개포주공1단지 40,145736.8,10166.7,16.507998,7.0,1,40,1982.0,0,0,...,1,40,1982.0,0,0,536710,0,4,37.483381,127.057068
2,서울특별시 강남구 개포동 개포주공5단지 60,136666.7,42678.6,0.000000,31.2,1,60,1983.0,0,0,...,1,60,1983.0,0,0,536710,0,4,37.487868,127.068988
3,서울특별시 강남구 개포동 개포주공5단지 80,177416.7,45000.0,0.000000,25.4,1,80,1983.0,0,0,...,1,80,1983.0,0,0,536710,0,4,37.487868,127.068988
4,서울특별시 강남구 개포동 개포주공6단지 50,125900.0,37909.1,9.134234,30.1,1,50,1983.0,0,0,...,1,50,1983.0,0,0,536710,0,4,37.488481,127.072502


In [19]:
# Big = pd.merge(data, dff, left_on='ID', right_on='ID', how='left')
# Big

,ID,거래금액(만원)_18/1,전세금(만원)_18/1,등락율18/1,전세가율18/1,인접역 수,전용면적,건축년도,Brand,BrandPower,...,연간소독횟수,개별난방TF,중앙난방TF,지역난방TF,복도식TF,혼합형TF,계단식TF,홈네트워크TF,관리시설수,부대복리시설수
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,42166.7,4.162633,32.7,0,80,1987.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,서울특별시 강남구 개포동 개포주공1단지 40,145736.8,10166.7,16.507998,7.0,1,40,1982.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,서울특별시 강남구 개포동 개포주공5단지 60,136666.7,42678.6,0.000000,31.2,1,60,1983.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,서울특별시 강남구 개포동 개포주공5단지 80,177416.7,45000.0,0.000000,25.4,1,80,1983.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,서울특별시 강남구 개포동 개포주공6단지 50,125900.0,37909.1,9.134234,30.1,1,50,1983.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,2500.0,6.000000,10.0,1,60,1998.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4087,서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,15000.0,0.000000,42.9,1,80,2005.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4088,서울특별시 중랑구 중화동 한신아파트 50,31756.2,24571.4,0.000000,77.4,5,50,1997.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4089,서울특별시 중랑구 중화동 한신아파트 60,36534.6,28111.1,2.996064,76.9,5,60,1997.1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
